In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')
import numpy as np
import networkqit as nq
import networkx as nx
import seaborn as sns
import numdifftools as nd
from numpy.random import random as rand
from networkqit import graph_laplacian as GL
from scipy.linalg import eigvalsh
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import pandas as pd
sns.set(font_scale=2)

In [ ]:
def ER(n,p):
    T=np.triu(rand([n,n]),1)
    T=T+T.T
    A=(T<p).astype(float)
    np.fill_diagonal(A,0)
    return A

def LER(n,p):
    return GL(ER(n,p))

In [ ]:
def graph_laplacian(A):
    return np.diag(A.sum(axis=0))-A

In [ ]:
N=100
pstar=0.2
A=ER(N,pstar)
L=graph_laplacian(A)
beta_range=np.logspace(0,-1,10)
M=nq.ErdosRenyi(N=len(A))
nx.density(nx.from_numpy_array(A))

In [ ]:
num_samples = 5
clip_gradients = None
max_iters = 1E3
eta = 1E-2
alpha = 1E-1
gtol = 1E-3
xtol = 1E-4

In [ ]:
solver = nq.Adam(A=A, x0=np.array([0.5,]),beta_range=beta_range)
solver.setup(expected_adj_fun=M, adj_sampling_fun=lambda x: ER(N,x), expected_laplacian_grad=M.expected_laplacian_grad)
sol = solver.run(num_samples=num_samples,clip_gradients=None,max_iters=max_iters, eta=eta, gtol=gtol)

In [ ]:
sns.set(font_scale=2)
df = pd.DataFrame(sol)
df['x'],df['beta']
fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(32,8))
ax[0].semilogx(df['T'],[x[0] for x in df['x'].values])
ax[0].set_xlabel('$1/\\beta$')
ax[0].set_ylabel('$p$')
ax[0].set_title('$p$')

ax[1].semilogx(df['T'],df.DeltaL.values)
ax[1].set_xlabel('$1/\\beta$')
ax[1].set_ylabel('$\Delta L$')
ax[1].set_title('$\Delta L$')